In [1]:
import cv2
import torch
from tracker import *
import numpy as np
count = set() # to store the ID of a person 
count2 = set()
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
cap=cv2.VideoCapture("C:/Users/ASUS/Downloads/project/highway.mp4")

def print_mouse_position(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        position = [x, y]
        print(position)
        
cv2.namedWindow('FRAME')
#cv2.setMouseCallback('FRAME', print_mouse_position) # to identify area by mouse event

tracker = Tracker()
area1 = [(448, 373), (521, 373), (521, 362), (448, 362)]
area2 = [(709, 382), (772, 382), (772, 368),(709, 368)]
while True:
    ret,frame=cap.read()
    frame=cv2.resize(frame, (1020, 500))
    cv2.polylines(frame,[np.array(area1, np.int32)], True, (0, 255, 0), 2)
    cv2.polylines(frame,[np.array(area2, np.int32)], True, (0, 0, 255), 2)
    results = model(frame)
    #frame = np.squeeze(results.render())
    points = [] # to store the coordinates of a person
    for index , row in results.pandas().xyxy[0].iterrows():
        x1, y1, x2, y2 = map(int, [row['xmin'], row['ymin'], row['xmax'], row['ymax']])
        n=(row['name'])
        if 'car' in n:
            points.append([x1, y1, x2, y2]) 
            #cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 2)
            #cv2.putText(frame, str(n), (x1, y1), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)

    boxes_id = tracker.update(points) # Make a unique ID for each person  
    #print(boxes_id)
    for box_id in boxes_id:
        x, y, w, h, idd = box_id       # (x,y)-> top left, (w,h)-> bottom right
        cv2.rectangle(frame,(x, y), (w, h), (255, 0, 255), 2)
        cv2.putText(frame,str(idd),(x,y),cv2.FONT_HERSHEY_PLAIN,3,(0,0,0),2)
        cv2.circle(frame, (w,h), 4,(255,0,0), -1)  # drew a circle in the bottom right of the car
        if cv2.pointPolygonTest(np.array(area1,np.int32), (w,h), False) >=0:
            count.add(idd)
        if cv2.pointPolygonTest(np.array(area2,np.int32), (w,h), False) >=0:
            count2.add(idd)
     
    sum = len(count)
    sum2 = len(count2)
    cv2.putText(frame,'number of cars in the green region is = '+str(sum), (50,65), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0),4)
    cv2.putText(frame,'number of cars in the red region is = '+str(sum2), (50,100), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255),4)
    cv2.imshow('FRAME',frame)
    if cv2.waitKey(25)&0xFF==27: # because frame rate for that video 15 frames/second
        break
cap.release()
cv2.destroyAllWindows()

Using cache found in C:\Users\ASUS/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-2-10 Python-3.11.5 torch-2.2.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
